In [68]:
#import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import glob
import imageio
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [61]:
def processData(imPath, labelPath):
    df1 = pd.read_csv(labelPath, header=0)
    yEncoder = OneHotEncoder(handle_unknown='ignore')
    yEncoder.fit(df1[['label']])
    #labelNames = yEncoder.get_feature_names()
    labels = yEncoder.transform(df1[['label']]).toarray()
    df2 = pd.DataFrame(data=labels)
    df1.drop(['label'], axis=1, inplace=True)
    df = pd.concat([df1, df2], axis=1)
    record = df.set_index('id').T.to_dict('list')
    #print(record[3])
    data = []
    imPaths = glob.glob(imPath)
    for i in imPaths:
        #print(i.split('.')[1][7:])
        ID = int(i.split('.')[1][7:])
        tmp = imageio.imread(i)
        '''
        r = tmp[:,:,0]
        g = tmp[:,:,1]
        b = tmp[:,:,2]
        rgb=np.array([r, g, b])
        '''
        tmp = [tmp, record[ID]]
        data.append(tmp)
    return data

In [62]:
df = pd.read_csv('./trainLabels.csv', header=0)
df[['label']]
data = processData('./train/*.png', './trainLabels.csv')
#imPaths = glob.glob('./train/*.png')
#imPaths = sorted(imPaths, key=lambda x:x[10:-3])
#print(imPaths)
#images = np.array([])
#images = []
#for i in imPaths:
    #images.append(imageio.imread(i))
    #images = np.append(images, imageio.imread(i))
#print(len(images))
#print(images.shape)

In [63]:
train, test = train_test_split(data, test_size=0.2)
trainX = np.array([i[0] for i in train])
trainY = np.array([i[1] for i in train])
testX = np.array([i[0] for i in test])
testY = np.array([i[1] for i in test])

In [75]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_62 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_60 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 8, 8, 32)          128       
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 8, 8, 32)        

In [76]:
model.fit(trainX, trainY, batch_size=50, epochs=20, verbose=1)

Train on 40000 samples
Epoch 1/20
40000/40000 [==============================] - 4s 94us/sample - loss: 1.3559 - accuracy: 0.5140
Epoch 2/20
40000/40000 [==============================] - 3s 75us/sample - loss: 0.9800 - accuracy: 0.6566
Epoch 3/20
40000/40000 [==============================] - 3s 75us/sample - loss: 0.8223 - accuracy: 0.7100
Epoch 4/20
40000/40000 [==============================] - 3s 77us/sample - loss: 0.7285 - accuracy: 0.7445
Epoch 5/20
40000/40000 [==============================] - 3s 75us/sample - loss: 0.6561 - accuracy: 0.7714
Epoch 6/20
40000/40000 [==============================] - 3s 75us/sample - loss: 0.5944 - accuracy: 0.7887
Epoch 7/20
40000/40000 [==============================] - 3s 75us/sample - loss: 0.5385 - accuracy: 0.8103
Epoch 8/20
40000/40000 [==============================] - 3s 75us/sample - loss: 0.4885 - accuracy: 0.8262
Epoch 9/20
40000/40000 [==============================] - 3s 77us/sample - loss: 0.4485 - accuracy: 0.8411
Epoch 10/20
40

In [77]:
model.evaluate(testX,  testY)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.053170090341568, 0.7429]